# COGS 108 - Data Checkpoint

# Names

- Sharon Chen
- Pamela Ghag
- Yuzi Chu
- Cheng Chang
- Stanley Hahm

<a id='research_question'></a>
# Research Question

Are there significant differences among courses under the Humanities, Social Sciences, and STEM departments at UCSD in terms of the correlation between course difficulty and teachers’ ratings on student-feedback platforms (e.g. CAPES, RateMyProfessor)?

# Dataset(s)

#### Dataset Name: CAPE

Link to the dataset: https://raw.githubusercontent.com/dcao/seascape/master/data/data.csv

Number of observations: 51281 rows(before cleanup) 

Description: The CAPE dataset includes information on the professor teaching the course, the course, the course date, average GPA received, percentage of students that recommended the professor, average GPA expected, the field of study the course is in. The raw dataset includes observations from Fall Quarter 2007 up to and including Winter Quarter 2020. 
 
Each observation has the following columns:

- Course Date

- Course ID

- Course Category (field of study)

- Level of Difficulty

- Professor Rating

- Grade Received 
 
We will be comparing the level of difficulty up against the professor rating to find out if there is correlation between the two depending on which field of study the course is categorized into. As we are measuring course difficulty in terms of average GPA (e.g higher average GPA = lower difficulty, lower average GPA = higher difficulty) we decided to remove any observations that may contain a null value in the average GPA column as they would not contribute to helping us reach a conclusion.
  


# Setup

In [1]:
## YOUR CODE HERE
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Configure libraries
# The seaborn library makes plots look nicer
sns.set(context = 'talk', style='white')

# Round decimals when displaying DataFrames
pd.set_option('precision', 2)

# Make plots just slightly bigger for displaying well in notebook
# set plotting size parameter
#plt.rcParams['figure.figsize'] = (10, 5)

# Data Cleaning

Describe your data cleaning steps here.

In [2]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION
df = pd.read_csv("https://raw.githubusercontent.com/dcao/seascape/master/data/data.csv")
#drop evaluation made for term WI20 and SP20
df = df.drop(df[(df.term =='WI20') | (df.term =='SP20')].index)

In [3]:
df

,instr,course,term,enrolled,evals,recClass,recInstr,hours,gpaExp,gpaAvg
0,"Butler, Elizabeth Annette",AAS 10,FA20,65,29,89.0,96.0,4.50,3.77,3.33
1,"Puritty, Chandler Elizabeth",AAS 190,FA20,19,5,100.0,100.0,2.10,4.00,NaN
2,"Andrews, Abigail Leslie",AIP 197T,FA20,34,11,100.0,100.0,4.06,3.67,NaN
3,"Jones, Ian William Nasser",ANAR 120,FA20,15,4,100.0,100.0,2.50,3.50,NaN
4,"Smith, Neil Gordon",ANAR 121,FA20,17,6,100.0,100.0,6.50,4.00,NaN
...,...,...,...,...,...,...,...,...,...,...
51276,"Li, Huai",VIS 105D,SU07,17,13,100.0,100.0,2.33,4.00,NaN
51277,"Guerrero, Raul M.",VIS 106A,SU07,16,14,92.0,92.0,7.21,3.79,NaN
51278,"Mangolte, Babette",VIS 194S,SU07,80,57,74.0,26.0,4.27,3.21,NaN
51279,"Holland, Nicole Murphy",VIS 22,SU07,40,33,100.0,96.0,4.32,3.62,NaN


In [4]:
df.shape

(48645, 10)

In [5]:
df.describe()

,enrolled,evals,recClass,recInstr,hours,gpaExp,gpaAvg
count,48645.00,48645.00,48645.00,48645.00,48645.00,47295.00,34282.00
mean,76.62,39.25,88.73,88.99,5.61,3.52,3.23
std,86.22,47.60,12.63,14.63,2.58,0.32,0.40
min,1.00,3.00,0.00,0.00,0.00,1.33,1.21
25%,20.00,10.00,83.00,84.00,4.00,3.30,2.93
50%,40.00,20.00,92.00,94.00,5.30,3.50,3.22
75%,103.00,50.00,100.00,100.00,6.93,3.75,3.52
max,1064.00,509.00,100.00,100.00,20.50,4.00,4.00


# Project Proposal (updated)

| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 1/20  |  1 PM | Read & Think about COGS 108 expectations; brainstorm topics/questions  | Determine best form of communication; Discuss and decide on final project topic; discuss hypothesis; begin background research | 
| 1/26  |  10 AM |  Do background research on topic | Discuss ideal dataset(s) and ethics; draft project proposal | 
| 2/1  | 10 AM  | Edit, finalize, and submit proposal; Search for datasets  | Discuss Wrangling and possible analytical approaches; Assign group members to lead each specific part   |
| 2/14  | 6 PM  | Import & Wrangle Data (Ant Man); EDA (Hulk) | Review/Edit wrangling/EDA; Discuss Analysis Plan   |
| 2/23  | 12 PM  | Finalize wrangling/EDA; Begin Analysis (Iron Man; Thor) | Discuss/edit Analysis; Complete project check-in |
| 3/13  | 12 PM  | Complete analysis; Draft results/conclusion/discussion (Wasp)| Discuss/edit full project |
| 3/19  | Before 11:59 PM  | NA | Turn in Final Project & Group Project Surveys |